### 可视化函数
- 需要安装open3d: pip3 install open3d

In [90]:
import numpy as np
import open3d as o3d
import random

def visualize_samples(d_sample):
    """
    d_sample: numpy array [10000, 3]
    """
    print("Points in downsample set: ", len(d_sample))
    source = o3d.geometry.PointCloud()
    source.points = o3d.utility.Vector3dVector(d_sample)
    color = [102.0 / 255.0 ,111.0 / 255.0, 142.0 / 255.0]
    source.paint_uniform_color(color)
    o3d.visualization.draw_geometries([source])

### 提取并可视化

In [2]:
# 展示原始点云
root = r'/home/haruki/下载/modelnet40_normal_resampled/chair/chair_0039.txt'
point_set = np.loadtxt(root, delimiter=',').astype(np.float32)[:, 0:3]
visualize_samples(point_set)

Points in downsample set:  10000


### 保存点云txt文件

In [61]:
import time
import os

def save_txt(d_sample):
    """
    d_sample: sampled point set
    可以根据运行的本地时间直接在当前文件夹下面创建并保存一个新txt文件，稍后根据自己需要重新命名
    """
    now = time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime())
    now_txt = now + '.txt'
    current_file_path = os.getcwd()
    txt_file_name = os.path.join(current_file_path, now_txt)
    np.savetxt(txt_file_name, d_sample, fmt='%.6e', delimiter=',')
    print("Text Saved!")

In [62]:
save_txt(point_set)

Text Saved!


### 保存点云截图png文件

In [87]:
def get_screen_filename():
    now = time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime())
    now_png = now + '.png'
    current_file_path = os.getcwd()
    png_file_name = os.path.join(current_file_path, now_png)
    return png_file_name


def custom_draw_geometry_with_key_callback(pcd):
    def close_window(vis):
        vis.close()
        return False

    def rotate_view(vis):
        ctr = vis.get_view_control()
        # every step how much to rotate
        # x -800
        # y 100
        ctr.rotate(-800.0, 100.0)
        return False

    def save_capture_screen_image(vis):
        vis.capture_screen_image(get_screen_filename())
        return False

    key_to_callback = {}
    key_to_callback[ord("C")] = close_window
    key_to_callback[ord("R")] = rotate_view
    key_to_callback[ord("S")] = save_capture_screen_image

    o3d.visualization.draw_geometries_with_key_callbacks([pcd], key_to_callback)
    
    
def visualization(point_set):
    source = o3d.geometry.PointCloud()
    source.points = o3d.utility.Vector3dVector(point_set)
    color = [102.0 / 255.0, 111.0 / 255.0, 142.0 / 255.0]
    source.paint_uniform_color(color)
    custom_draw_geometry_with_key_callback(source)

In [91]:
visualization(point_set)

- 感觉可以的： chair_0005,chair_0020,chair_0039
- 最后选择0039，感觉这个比较好
- 上传chair_0039.txt到GitHub项目中的data文件夹

### Subsample: FPS算法

In [3]:
def fps(original, npoints):
    """
    original: numpy array [10000, 3]
    npoints: output point number
    return: downsampled point set [npoints, 3]
    """
    center_xyz = np.sum(original, 0)
    center_xyz = center_xyz / len(original)
    dist = np.sum((original - center_xyz) ** 2, 1)
    farthest = np.argmax(dist)
    distance = np.ones(len(original)) * 1e10
    target_index = np.zeros(npoints, dtype=np.int32)

    for i in range(npoints):
        target_index[i] = farthest
        target_point_xyz = original[target_index[i], :]
        
        dist = np.sum((original - target_point_xyz) ** 2, 1)
        mask = dist < distance
        distance[mask] = dist[mask]
        farthest = np.argmax(distance)
    
    return original[target_index]

In [4]:
# 展示下采样之后的点云
downsample_fps = fps(point_set, 1024)
visualize_samples(downsample_fps)

Points in downsample set:  1024


In [92]:
visualization(downsample_fps)

### Augmentation: PointWOLF算法

In [13]:
import numpy as np


class PointWOLF(object):
    def __init__(self, w_sigma):
        self.num_anchor = 4
        self.sample_type = 'fps'  # 'random'
        self.sigma = w_sigma

        self.R_range = (-abs(10), abs(10))
        self.S_range = (1., 3)
        self.T_range = (-abs(0.25), abs(0.25))

    def __call__(self, pos):
        M = self.num_anchor  # (Mx3)
        N, _ = pos.shape  # (N)

        if self.sample_type == 'random':
            idx = np.random.choice(N, M)  # (M)
        elif self.sample_type == 'fps':
            idx = self.fps(pos, M)  # (M)

        pos_anchor = pos[idx]  # (M,3), anchor point

        pos_repeat = np.expand_dims(pos, 0).repeat(M, axis=0)  # (M,N,3)
        pos_normalize = np.zeros_like(pos_repeat, dtype=pos.dtype)  # (M,N,3)
        pos_normalize = pos_repeat - pos_anchor.reshape(M, -1, 3)

        # Local transformation at anchor point
        pos_transformed = self.local_transformaton(pos_normalize)  # (M,N,3)

        # Move to origin space
        pos_transformed = pos_transformed + pos_anchor.reshape(M, -1, 3)  # (M,N,3)

        pos_new = self.kernel_regression(pos, pos_anchor, pos_transformed)
        pos_new = self.normalize(pos_new)

        return pos_new.astype('float32')

    def kernel_regression(self, pos, pos_anchor, pos_transformed):
        M, N, _ = pos_transformed.shape

        # Distance between anchor points & entire points
        sub = np.expand_dims(pos_anchor, 1).repeat(N, axis=1) - np.expand_dims(pos, 0).repeat(M, axis=0)  # (M,N,3), d

        project_axis = self.get_random_axis(1)

        projection = np.expand_dims(project_axis, axis=1) * np.eye(3)  # (1,3,3)

        # Project distance
        sub = sub @ projection  # (M,N,3)
        sub = np.sqrt(((sub) ** 2).sum(2))  # (M,N)

        # Kernel regression
        weight = np.exp(-0.5 * (sub ** 2) / (self.sigma ** 2))  # (M,N)
        pos_new = (np.expand_dims(weight, 2).repeat(3, axis=-1) * pos_transformed).sum(0)  # (N,3)
        pos_new = (pos_new / weight.sum(0, keepdims=True).T)  # normalize by weight
        return pos_new

    def fps(self, pos, npoint):
        N, _ = pos.shape
        centroids = np.zeros(npoint, dtype=np.int_)  # (M)
        distance = np.ones(N, dtype=np.float64) * 1e10  # (N)
        farthest = np.random.randint(0, N, (1,), dtype=np.int_)
        for i in range(npoint):
            centroids[i] = farthest
            centroid = pos[farthest, :]
            dist = ((pos - centroid) ** 2).sum(-1)
            mask = dist < distance
            distance[mask] = dist[mask]
            farthest = distance.argmax()
        return centroids

    def local_transformaton(self, pos_normalize):
        M, N, _ = pos_normalize.shape
        transformation_dropout = np.random.binomial(1, 0.5, (M, 3))  # (M,3)
        transformation_axis = self.get_random_axis(M)  # (M,3)

        degree = np.pi * np.random.uniform(*self.R_range, size=(M, 3)) / 180.0 * transformation_dropout[:,
                                                                                 0:1]  # (M,3), sampling from (-R_range, R_range)

        scale = np.random.uniform(*self.S_range, size=(M, 3)) * transformation_dropout[:,
                                                                1:2]  # (M,3), sampling from (1, S_range)
        scale = scale * transformation_axis
        scale = scale + 1 * (scale == 0)  # Scaling factor must be larger than 1

        trl = np.random.uniform(*self.T_range, size=(M, 3)) * transformation_dropout[:,
                                                              2:3]  # (M,3), sampling from (1, T_range)
        trl = trl * transformation_axis

        # Scaling Matrix
        S = np.expand_dims(scale, axis=1) * np.eye(3)  # scailing factor to diagonal matrix (M,3) -> (M,3,3)
        # Rotation Matrix
        sin = np.sin(degree)
        cos = np.cos(degree)
        sx, sy, sz = sin[:, 0], sin[:, 1], sin[:, 2]
        cx, cy, cz = cos[:, 0], cos[:, 1], cos[:, 2]
        R = np.stack([cz * cy, cz * sy * sx - sz * cx, cz * sy * cx + sz * sx,
                      sz * cy, sz * sy * sx + cz * cy, sz * sy * cx - cz * sx,
                      -sy, cy * sx, cy * cx], axis=1).reshape(M, 3, 3)

        pos_normalize = pos_normalize @ R @ S + trl.reshape(M, 1, 3)
        return pos_normalize

    def get_random_axis(self, n_axis):
        axis = np.random.randint(1, 8, (
            n_axis))  # 1(001):z, 2(010):y, 3(011):yz, 4(100):x, 5(101):xz, 6(110):xy, 7(111):xyz
        m = 3
        axis = ((axis[:, None] & (1 << np.arange(m))) > 0).astype(int)
        return axis

    def normalize(self, pos):
        pos = pos - pos.mean(axis=-2, keepdims=True)
        scale = (1 / np.sqrt((pos ** 2).sum(1)).max()) * 0.999999
        pos = scale * pos
        return pos


In [31]:
# 初始化点云函数
# 不需要修改参数，只是重新执行这个函数就可以得到不同的增强结果
augment_func = PointWOLF(0.4)

In [32]:
# 生成augmentation
aug_point_set = augment_func(point_set)

In [33]:
# 可视化经过增强处理的点云
visualize_samples(aug_point_set)

Points in downsample set:  10000


In [93]:
visualization(aug_point_set)

In [ ]:
# 对比原始点云的样子
visualize_samples(point_set)

### Crop_Method 1/3  Slice

In [38]:
# 3 slices per axis， 40% points of one point cloud
def get_index(index_slice, ration_per_slice, overlap_ration, num_all_points):
    # index_slice = 0, 1, 2
    start_index = index_slice * (ration_per_slice - overlap_ration) * num_all_points
    end_index = start_index + ration_per_slice * num_all_points
    return int(start_index), int(end_index)

def get_slice(point_set, npoints, xyz_dim, index_slice):
    # xyz_dim: 0, 1, 2 for x, y, z
    axis = 'x'
    if xyz_dim == 1:
        axis = 'y'
    elif xyz_dim == 2:
        axis = 'z'
    print(f"这个是沿着{axis}轴来切割的，第{index_slice}个slice.")
    
    start_index, end_index = get_index(index_slice, 0.4, 0.1, len(point_set))
    big_patch_index = np.argsort(point_set, axis=0)[start_index: end_index, xyz_dim]
    big_patch = point_set[big_patch_index]
    random.shuffle(big_patch)
    # 返回fps后的值，fps前的值
    return (fps(big_patch, npoints), big_patch)

In [39]:
after_fps_slice, direct_slice = get_slice(point_set, 1024, 0, 0)

这个是沿着x轴来切割的，第0个slice.


In [40]:
# fps之后的点云可视化
visualize_samples(after_fps_slice)

Points in downsample set:  1024


In [95]:
visualization(after_fps_slice)

In [41]:
# fps之前的点云
visualize_samples(direct_slice)

Points in downsample set:  4000


In [96]:
visualization(direct_slice)

### Crop_Method 2/3 Cube

In [43]:
def get_random_center():
    """随机生成一个cube中心点"""
    u = np.random.uniform(-1.0, 1.0)
    theta = 2 * np.pi * np.random.uniform(0.0, 2)
    
    x = np.power((1 - u * u), 1/2) * np.cos(theta)
    x = np.abs(x)
    x = np.random.uniform(-x, x)
    y = np.power((1 - u * u), 1/2) * np.sin(theta)
    y = np.abs(y)
    y = np.random.uniform(-y, y)
    z = u
    return (x, y, z)


def point_in_cube(point_xyz, center_xyz, side_length):
    """判断一个点是否在cube内部"""
    flag = True
    for i in range(0, len(point_xyz)):
        if abs(point_xyz[i] - center_xyz[i]) >= (side_length / 2):
            flag = False
            break
    return flag


def get_1_cube(point_set, center_xyz, side_length, npoints=1024):
    """从单个点云中，根据1个中心点找到1个cube"""
    output_samples = []
    for i in range(0, len(point_set)):
        if point_in_cube(point_set[i], center_xyz, side_length):
            output_samples.append(i)
    samples = point_set[output_samples]
    
    if len(output_samples) > npoints:
        result = fps(samples, npoints)
        return result
    else:
        return get_1_cube(point_set, center_xyz, side_length + 0.2, npoints)
    
    
def get_cubes(point_set, num_cube=8, side_length=0.5, npoints=1024):
    """从单个点云中，根据8个中心点找到8个cube"""
    result = np.ones((num_cube, npoints, 3))
    centers = []  # record 8 cube centers
    for i in range(0, num_cube):
        center = get_random_center()
        centers.append(center)
        result[i] = get_1_cube(point_set, center, side_length, npoints)
    return result, centers

In [45]:
cubes, centers = get_cubes(point_set)
print(cubes.shape)

(8, 1024, 3)


In [47]:
# 查看其中一个cube
random_choice = 5
chose_1_cube = cubes[random_choice, :, :]
visualize_samples(chose_1_cube)

Points in downsample set:  1024


In [97]:
visualization(chose_1_cube)

### Crop_Method 3/3 KNN(Sphere)

In [55]:
import torch

def index_points(points, idx):
    """
    Input:
        points: input points data, [B, N, C]
        idx: sample index data, [B, S, [K]]
    Return:
        new_points:, indexed points data, [B, S, [K], C]
    """
    raw_size = idx.size()
    idx = idx.reshape(raw_size[0], -1)
    res = torch.gather(points, 1, idx[..., None].expand(-1, -1, points.size(-1)))
    return res.reshape(*raw_size, -1)


def b_FPS(xyz, npoint):
    """
    Input:
        xyz: pointcloud data, [B, N, 3]
        npoint: number of samples
    Return:
        centroids: sampled pointcloud index, [B, npoint]
    """
    device = xyz.device
    B, N, C = xyz.shape
    centroids = torch.zeros(B, npoint, dtype=torch.long).to(device)
    distance = torch.ones(B, N).to(device)* 1e10
    farthest = torch.randint(0, N, (B,), dtype=torch.long).to(device)
    batch_indices = torch.arange(B, dtype=torch.long).to(device)
    for i in range(npoint):
        centroids[:, i] = farthest
        centroid = xyz[batch_indices, farthest, :].view(B, 1, 3)
        dist = torch.sum((xyz - centroid) ** 2, -1)
        distance = torch.min(distance, dist)
        farthest = torch.max(distance, -1)[1]
    return centroids, index_points(xyz, centroids)


def k_points(a, b, k):
    # a: small, b: big one
    inner = -2 * torch.matmul(a, b.transpose(2, 1))
    aa = torch.sum(a**2, dim=2, keepdim=True)
    bb = torch.sum(b**2, dim=2, keepdim=True)
    pairwise_distance = -aa - inner - bb.transpose(2, 1)
    idx = pairwise_distance.topk(k=k, dim=-1)[1]
    return idx


def new_k_patch(x, k=2048, n_patch=8, n_points=1024):
    """k = 2048, with FPS subsample method and FPS centroids"""
    patch_centers_index, _ = b_FPS(x, n_patch)  # torch.Size([B, n_patch])
    center_point_xyz = index_points(x, patch_centers_index)  # [B, n_patch]

    idx = k_points(center_point_xyz, x, k)  # B, k, 2048
    idx = idx.permute(0, 2, 1)  # B, k, n_patch

    new_patch = torch.zeros([n_patch, x.shape[0], n_points, x.shape[-1]]).to(device)
    for i in range(n_patch):
        patch_idx = idx[:, :, i].reshape(x.shape[0], -1)
        _, patch_points = b_FPS(index_points(x, patch_idx), n_points)
        new_patch[i] = patch_points
    new_patch = new_patch.permute(1, 0, 2, 3)   # torch.Size([B, 8, 1024, 3])
    return new_patch


def new_k_patch_1024(x, k=1024, n_patch=8, n_points=1024):
    """k=1024 without subsample method but FPS centroids"""
    patch_centers_index, _ = b_FPS(x, n_patch)  # torch.Size([B, n_patch])
    center_point_xyz = index_points(x, patch_centers_index)  # [B, n_patch]

    idx = k_points(center_point_xyz, x, k)  # B, n_patch, 1024
    idx = idx.permute(0, 2, 1)  # B, k, n_patch

    new_patch = torch.zeros([n_patch, x.shape[0], n_points, x.shape[-1]]).to(device)
    for i in range(n_patch):
        patch_idx = idx[:, :, i].reshape(x.shape[0], -1)
        patch_points = index_points(x, patch_idx)
        new_patch[i] = patch_points
    new_patch = new_patch.permute(1, 0, 2, 3)   # torch.Size([B, 8, 1024, 3])
    return new_patch


def random_k_patch_1024(x, k=1024, n_patch=8, n_points=1024):
    """k=1024 with random centroids"""
    def get_random_index(point_set, num):
        result = []
        for i in range(point_set.shape[0]):
            result.append(random.sample(range(0, point_set.shape[1]), num))
        return torch.tensor(result, dtype=torch.int64).to(device)
    
    patch_centers_index = get_random_index(x, n_patch)  # torch.Size([B, n_patch])
    center_point_xyz = index_points(x, patch_centers_index)  # [B, n_patch]

    idx = k_points(center_point_xyz, x, k)  # B, n_patch, 1024
    idx = idx.permute(0, 2, 1)  # B, k, n_patch

    new_patch = torch.zeros([n_patch, x.shape[0], n_points, x.shape[-1]]).cuda()
    for i in range(n_patch):
        patch_idx = idx[:, :, i].reshape(x.shape[0], -1)
        patch_points = index_points(x, patch_idx)
        new_patch[i] = patch_points
    new_patch = new_patch.permute(1, 0, 2, 3)   # torch.Size([B, 8, 1024, 3])
    return new_patch

In [56]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
tensor_point_set = torch.tensor(point_set[np.newaxis, :, :]).to(device)

In [57]:
knn_2048_fps_centroids = new_k_patch(tensor_point_set)
knn_1024_fps_centroids = new_k_patch_1024(tensor_point_set)
knn_1024_random_centroids = new_k_patch_1024(tensor_point_set)

In [58]:
knn_2048_fps_centroids.shape

torch.Size([1, 8, 1024, 3])

In [59]:
# 维度不同需要转换
knn_2048_fps_centroids = knn_2048_fps_centroids.reshape(8, 1024, 3)
k1 = knn_2048_fps_centroids.cpu().numpy()
k1.shape

(8, 1024, 3)

In [98]:
demo = k1[0]
visualization(demo)